In [2]:
from insightface.app import FaceAnalysis
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
from insightface.app import FaceAnalysis
import cv2
import os
import numpy as np

In [3]:
def calc_cost(embedding1, embedding2):
    return cosine_similarity([embedding1], [embedding2])[0][0]

In [4]:
app = FaceAnalysis(name="Buffalo_l",  providers=['CPUExecutionProvider'])
app.prepare(ctx_id=0, det_size=(640, 640))

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\ATUS/.insightface\models\Buffalo_l\1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\ATUS/.insightface\models\Buffalo_l\2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\ATUS/.insightface\models\Buffalo_l\det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\ATUS/.insightface\models\Buffalo_l\genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\ATUS/.insightface\models\Buffalo_l\w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127

In [5]:
def toembeddings(img):
    return app.get(img)

In [6]:
def init_embeddings(pathdata, testsize):
    # Đường dẫn tới bộ dữ liệu LFW
    # lfw_path = r"C:\Users\ATUS\Desktop\LUONG ANH TU\Database\lfw"
    # Lưu embeddings của tất cả các khuôn mặt
    embeddings = []
    labels = []

    # Lặp qua từng thư mục trong bộ dữ liệu LFW (mỗi thư mục là một người)
    for person_name in os.listdir(pathdata):
        if (testsize == 0): break
        person_path = os.path.join(pathdata, person_name)
        if os.path.isdir(person_path):
            # Lặp qua từng ảnh của người đó
            for img_name in os.listdir(person_path):
                if (testsize == 0): break
                img_path = os.path.join(person_path, img_name)
                # Đọc ảnh và lấy embedding từ mô hình
                img = cv2.imread(img_path)
                faces = toembeddings(img)
                if len(faces) > 0:
                    face_embedding = faces[0].embedding
                    print(f"{testsize} {person_name}")
                    embeddings.append(face_embedding)
                    labels.append(person_name)
                    testsize-=1
    return embeddings, labels

In [11]:
embeddings, labels = init_embeddings(r"C:\Users\ATUS\Desktop\LUONG ANH TU\Database\lfw", 1000)

c:\Users\ATUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\insightface\utils\transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4


1000 Aaron_Eckhart
999 Aaron_Guiel
998 Aaron_Patterson
997 Aaron_Peirsol
996 Aaron_Peirsol
995 Aaron_Peirsol
994 Aaron_Peirsol
993 Aaron_Pena
992 Aaron_Sorkin
991 Aaron_Sorkin
990 Aaron_Tippin
989 Abbas_Kiarostami
988 Abba_Eban
987 Abdel_Aziz_Al-Hakim
986 Abdel_Madi_Shabneh
985 Abdel_Nasser_Assidi
984 Abdel_Nasser_Assidi
983 Abdoulaye_Wade
982 Abdoulaye_Wade
981 Abdoulaye_Wade
980 Abdoulaye_Wade
979 Abdulaziz_Kamilov
978 Abdullah
977 Abdullah
976 Abdullah
975 Abdullah
974 Abdullah_Ahmad_Badawi
973 Abdullah_al-Attiyah
972 Abdullah_al-Attiyah
971 Abdullah_al-Attiyah
970 Abdullah_Gul
969 Abdullah_Gul
968 Abdullah_Gul
967 Abdullah_Gul
966 Abdullah_Gul
965 Abdullah_Gul
964 Abdullah_Gul
963 Abdullah_Gul
962 Abdullah_Gul
961 Abdullah_Gul
960 Abdullah_Gul
959 Abdullah_Gul
958 Abdullah_Gul
957 Abdullah_Gul
956 Abdullah_Gul
955 Abdullah_Gul
954 Abdullah_Gul
953 Abdullah_Gul
952 Abdullah_Gul
951 Abdullah_Nasseef
950 Abdullatif_Sener
949 Abdullatif_Sener
948 Abdul_Majeed_Shobokshi
947 Abdul_Rahman

In [12]:
def calculate_accuracy(embeddings, labels, threshold=0.4):
    correct = 0
    total = len(labels)
    count = 0
    # Duyệt qua tất cả các embedding và so sánh
    for i in range(total-1):
        for j in range(i + 1, total):
            count+=1
            tmpthresh = calc_cost(embeddings[i], embeddings[j])
            if (labels[i] == labels[j] and tmpthresh >= threshold): correct+=1
            if (labels[i] != labels[j] and tmpthresh < threshold): correct+=1
    # Tính Accuracy
    accuracy = correct / count
    return accuracy

In [18]:
def calculate_FPIR(embeddings, labels, threshold=0.4):
    correct = 0
    total = len(labels)
    count = 0
    # Duyệt qua tất cả các embedding và so sánh
    for i in range(total-1):
        for j in range(i + 1, total):
            
            if (labels[i] != labels[j]):
                count += 1
                tmpthresh = calc_cost(embeddings[i], embeddings[j])
                if (tmpthresh >= threshold): correct+=1
    return correct / count

def calculate_FNIR(embeddings, labels, threshold=0.4):
    correct = 0
    total = len(labels)
    count = 0
    # Duyệt qua tất cả các embedding và so sánh
    for i in range(total-1):
        for j in range(i + 1, total):
            if (labels[i] == labels[j]):
                count += 1
                tmpthresh = calc_cost(embeddings[i], embeddings[j])
                if (tmpthresh < threshold): correct+=1
    return correct / count



In [20]:
print(f"ACC: {calculate_accuracy(embeddings, labels)}")
print(f"FPIR: {calculate_FPIR(embeddings, labels)}")
print(f"FNIR: {calculate_FNIR(embeddings, labels)}")

ACC: 0.9994714714714714
FPIR: 2.031413782736233e-06
FNIR: 0.036366150442477874
